In [1]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import random

df = pd.read_csv('D:/sql/Data set/NLP/train.csv')

X = df['text']
y = df['target']

import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
corpus = []
for i in range(0, len(X)):
    review = re.sub('[^a-zA-Z]', ' ', X[i])
    review = review.lower()
    review = nltk.word_tokenize(review)
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    corpus.append(review)
    
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray()
print(len(cv.get_feature_names_out()))
pd.DataFrame(X,columns=cv.get_feature_names_out())

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=42)


18868


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, classification_report, ConfusionMatrixDisplay, confusion_matrix
from xgboost import XGBClassifier
import xgboost as xgb

In [30]:

classifier_rf = XGBClassifier(eta = 0.3,max_depth=80,n_estimators=66,gamma =0,
                                       random_state = 32)
classifier_rf.fit(X_train, y_train)
predictions = classifier_rf.predict(X_test)

from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

D:\Anaconda_3\Lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


[[728 146]
 [184 465]]


              precision    recall  f1-score   support

           0       0.80      0.83      0.82       874
           1       0.76      0.72      0.74       649

    accuracy                           0.78      1523
   macro avg       0.78      0.77      0.78      1523
weighted avg       0.78      0.78      0.78      1523



In [ ]:
classifier_rfe = RandomForestClassifier(n_estimators = 22, criterion = 'entropy',max_depth=175,
                                       random_state = 0)
classifier_rfe.fit(X_train, y_train)
predictions = classifier_rfe.predict(X_test)

from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

In [31]:
df_test = pd.read_csv("D:/sql/Data set/NLP/test.csv")
sample_submission = pd.read_csv("D:/sql/Data set/NLP/sample_submission.csv")

In [32]:
X_=df_test['text']

In [33]:
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
corpus_1 = []
for i in range(0, len(X_)):
    review = re.sub('[^a-zA-Z]', ' ', X_[i])
    review = review.lower()
    review = nltk.word_tokenize(review)
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    corpus_1.append(review)

In [34]:
ss= cv.fit_transform(corpus_1).toarray()
print(len(cv.get_feature_names_out()))
ss_X=pd.DataFrame(ss,columns=cv.get_feature_names_out())

10765


In [35]:
10765+8103

18868

In [36]:
num_columns_to_add = 8103

# Create a DataFrame with zero values
zero_data = {
    f'ZeroColumn{i}': [0] * len(ss_X) for i in range(1, num_columns_to_add + 1)
}

zero_df = pd.DataFrame(zero_data)

# Concatenate the original DataFrame and the zero DataFrame along the columns (axis=1)
result_df = pd.concat([ss_X, zero_df], axis=1)

In [37]:
result_df.shape

(3263, 18868)

In [38]:
X_TEST=result_df.values

In [39]:
X_TEST.shape

(3263, 18868)

In [40]:
sample_submission["target"] = classifier_rf.predict(X_TEST)

In [41]:
sample_submission["target"].value_counts()

target
0    3152
1     111
Name: count, dtype: int64

In [42]:
sample_submission.to_csv("submission.csv", index=False)